In [1]:
import pandas as pd
import mitosheet

This notebook cleans csv data in order to be ready for sql database.

# Clean Geolocation Data

Clean geolocation_city as there are multiple spellings of some cities, containing portuguese spellings aswell as english spellings. Keep city spellings consistent with english spelling.

In [2]:
geo_data = pd.read_csv('olist_geolocation_dataset.csv/olist_geolocation_dataset.csv')

In [3]:
geo_data.head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP


In [10]:
geo_data.shape

(1000163, 5)

Group the zip code prefixes and take the average latitude/longditude coordinates for each zip code. This will make data aggregation better in the database as we dont require increased granularity.

In [40]:
# group geolocation data by zip code prefix, avg lat lng coords
geo_loc_grouped = geo_data.groupby('geolocation_zip_code_prefix').agg({'geolocation_lat': 'mean', 'geolocation_lng': 'mean'})
# join initial data to grouped geo data
geo_loc_joined = geo_loc_grouped.join(geo_data.set_index('geolocation_zip_code_prefix'), on='geolocation_zip_code_prefix', how='left', lsuffix='_df1', rsuffix='_df2')
# reset index
geo_loc_joined = geo_loc_joined.reset_index()
# keep neccessary cols, drop duplicates zip codes
geo_loc_clean = geo_loc_joined.iloc[:,[0,1,2,5,6]].drop_duplicates(subset='geolocation_zip_code_prefix', keep='first')
# change columns names
geo_loc_clean.columns = ['geolocation_zip_code_prefix','geolocation_lat', 'geolocation_lng', 'geolocation_city', 'geolocation_state']
# reset index
geo_loc_clean = geo_loc_clean.reset_index(drop=True)
# save to csv
geo_loc_clean.to_csv('olist_geolocation_dataset.csv/olist_geoloc_clean.csv', index=False, encoding='utf-8')

In [97]:
clean_geo_data = pd.read_csv('olist_geolocation_dataset.csv/olist_geoloc_clean.csv')

Replace special characters with english characters.

In [98]:
## clean geolocation_city
# replace ã with a
def replace_ã(s):
    return s.replace('ã', 'a')
clean_geo_data['geolocation_city'] = list(map(replace_ã, clean_geo_data['geolocation_city']))
# replace ç with c
def replace_ç(s):
    return s.replace('ç', 'c')
clean_geo_data['geolocation_city'] = list(map(replace_ç, clean_geo_data['geolocation_city']))
# replace é with e
def replace_é(s):
    return s.replace('é', 'e')
clean_geo_data['geolocation_city'] = list(map(replace_é, clean_geo_data['geolocation_city']))
# replace õ with o
def replace_õ(s):
    return s.replace('õ', 'o')
clean_geo_data['geolocation_city'] = list(map(replace_õ, clean_geo_data['geolocation_city']))
# replace í with i
def replace_í(s):
    return s.replace('í', 'i')
clean_geo_data['geolocation_city'] = list(map(replace_í, clean_geo_data['geolocation_city']))
# replace á with a
def replace_á(s):
    return s.replace('á', 'a')
clean_geo_data['geolocation_city'] = list(map(replace_á, clean_geo_data['geolocation_city']))
# replace ó with o
def replace_ó(s):
    return s.replace('ó', 'o')
clean_geo_data['geolocation_city'] = list(map(replace_ó, clean_geo_data['geolocation_city']))
# replace â with a
def replace_â(s):
    return s.replace('â', 'a')
clean_geo_data['geolocation_city'] = list(map(replace_â, clean_geo_data['geolocation_city']))
# replace - with ' '
def replace_hyphon(s):
    return s.replace('-', ' ')
clean_geo_data['geolocation_city'] = list(map(replace_hyphon, clean_geo_data['geolocation_city']))
# replace ê with e
def replace_ê(s):
    return s.replace('ê', 'e')
clean_geo_data['geolocation_city'] = list(map(replace_ê, clean_geo_data['geolocation_city']))
# replace ú with u
def replace_ú(s):
    return s.replace('ú', 'u')
clean_geo_data['geolocation_city'] = list(map(replace_ú, clean_geo_data['geolocation_city']))
# replace ü with u
def replace_ü(s):
    return s.replace('ü', 'u')
clean_geo_data['geolocation_city'] = list(map(replace_ü, clean_geo_data['geolocation_city']))


In [9]:
# replace 'rio de janeiro, rio de janeiro, brasil' with 'rio de janeiro'
mask_idx = clean_geo_data.query("geolocation_city == 'rio de janeiro, rio de janeiro, brasil'").index
clean_geo_data.loc[mask_idx, 'geolocation_city'] = 'rio de janeiro'

In [13]:
# save data
clean_geo_data.to_csv('olist_geolocation_dataset.csv/olist_geoloc_clean.csv', index=False, encoding='utf-8')

# Clean Products Data

Clean product_name_category as category names are in portuguese, we will translate these into english.

In [57]:
# load product data
prod_data = pd.read_csv('olist_products_dataset.csv/olist_products_dataset.csv')

In [58]:
# load translation data
trans_data = pd.read_csv('product_category_name_translation.csv')

In [22]:
prod_data

,product_id,product_category_name,product_name_length,product_description_length,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0
...,...,...,...,...,...,...,...,...,...
32946,a0b7d5a992ccda646f2d34e418fff5a0,moveis_decoracao,45.0,67.0,2.0,12300.0,40.0,40.0,40.0
32947,bf4538d88321d0fd4412a93c974510e6,construcao_ferramentas_iluminacao,41.0,971.0,1.0,1700.0,16.0,19.0,16.0
32948,9a7c6041fa9592d9d9ef6cfe62a71f8c,cama_mesa_banho,50.0,799.0,1.0,1400.0,27.0,7.0,27.0
32949,83808703fc0706a22e264b9d75f04a2e,informatica_acessorios,60.0,156.0,2.0,700.0,31.0,13.0,20.0


In [27]:
trans_data

,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories
2,automotivo,auto
3,cama_mesa_banho,bed_bath_table
4,moveis_decoracao,furniture_decor
...,...,...
66,flores,flowers
67,artes_e_artesanato,arts_and_craftmanship
68,fraldas_higiene,diapers_and_hygiene
69,fashion_roupa_infanto_juvenil,fashion_childrens_clothes


In [59]:
# translate product names
for i in range(len(trans_data)):
    prod_name = trans_data['product_category_name'][i] # brazilian name
    eng_prod_name = trans_data['product_category_name_english'][i] # english translation
    # get indices of product
    prod_idx = prod_data[prod_data['product_category_name'] == prod_name].index
    # translate product
    prod_data.loc[prod_idx, 'product_category_name'] = eng_prod_name

In [65]:
# check for duplicates
prod_data.duplicated().sum()

0

In [24]:
# check primary key for duplicates
products_data['product_id'].duplicated().sum()

0

In [61]:
# save data
prod_data.to_csv('olist_products_dataset.csv/olist_products_clean.csv', index=False, encoding='utf-8')

# clean customers data

Remove zip codes from customers data that are not in geolocation data so that primary and foreign keys align.

In [3]:
# load data
geo_data = pd.read_csv('olist_geolocation_dataset.csv/olist_geolocation_dataset.csv')
customer_data = pd.read_csv('olist_customers_dataset.csv/olist_customers_dataset.csv')

In [18]:
## zip codes not in geolocation data
non_inc = []
for i in range(len(customer_data)):
    if customer_data['customer_zip_code_prefix'][i] not in geo_data['geolocation_zip_code_prefix'].values:
        non_inc.append(customer_data['customer_zip_code_prefix'][i])

In [40]:
# indices to drop of zip codes not in geolocation data
idx_to_drop = []
for zip_code in non_inc:
    c_idx = list(customer_data.query(f"customer_zip_code_prefix == {zip_code}").index)
    for idx in c_idx:
        if idx in idx_to_drop:
            continue
        else:
            idx_to_drop.append(idx)

In [43]:
# drop and save
customer_data.drop(idx_to_drop, inplace=True)
customer_data.to_csv('olist_customers_dataset.csv/olist_customers_dataset_dropped_zips.csv', index=False, encoding='utf-8')

# Clean Orders Data

In [2]:
### check foreign key for customers_id on orders data
orders_data = pd.read_csv('olist_orders_dataset.csv\olist_orders_dataset.csv')
customers_data = pd.read_csv('olist_customers_dataset.csv\olist_customers_dataset_dropped_zips.csv')

In [16]:
# check primary key for duplicates
orders_data['order_id'].duplicated().sum()

0

Remove order data customer id's that are not in customer data customer id's so that primary and foreign keys align.

In [3]:
# get customer id's in orders data that are not in customers data
no_id = [x for x in orders_data['customer_id'].values if x not in customers_data['customer_id'].values]

In [6]:
# remove order data customer id's that are not in customer data customer id's
idx_to_drop = []
for c_id in no_id:
    cid_idx = list(orders_data.query(f"customer_id == '{c_id}'").index)
    for idx in cid_idx:
        if idx in idx_to_drop:
            continue
        else:
            idx_to_drop.append(idx)

In [9]:
# drop id's and save data
orders_data.drop(idx_to_drop, inplace=True)
orders_data.to_csv('olist_orders_dataset.csv/olist_orders_dataset_dropped_ids.csv', index=False, encoding='utf-8')

Clean nan values.

In [2]:
### clean nan values in datetime columns!!!!
orders_data = pd.read_csv('olist_orders_dataset.csv/olist_orders_dataset_dropped_ids.csv')

In [11]:
# check for nan values
orders_data.isnull().sum()

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 159
order_delivered_carrier_date     1776
order_delivered_customer_date    2951
order_estimated_delivery_date       0
dtype: int64

In [32]:
# fill nan values with 01-01-9999 00:00:00 just to signify no value
orders_data.fillna(value='9999-01-01 00:00:00', inplace=True)

In [35]:
# save data
orders_data.to_csv('olist_orders_dataset.csv\olist_orders_clean_dataset.csv', index=False, encoding='utf-8')

# Clean Order Payments Data

In [28]:
# load data
order_pay_data = pd.read_csv('olist_order_payments_dataset.csv\olist_order_payments_dataset.csv')

In [31]:
# check shape
order_pay_data.shape

(103886, 5)

In [30]:
# check primary key for duplicates
order_pay_data['order_id'].duplicated().sum()

4446

In [35]:
# remove duplicated order ids
pay_dup_idx = order_pay_data[order_pay_data['order_id'].duplicated()].index
order_pay_data.drop(pay_dup_idx, inplace=True)
order_pay_data.to_csv('olist_order_payments_dataset.csv\olist_order_payments_dataset_dropped_dups.csv')

In [37]:
# check foreign key alignment, order id in order payments must match order id in orders
# load orders data
orders_data = pd.read_csv('olist_orders_dataset.csv\olist_orders_clean_dataset.csv')
# check for mismatch order ids
non_inc_ids = [x for x in order_pay_data['order_id'].values if x not in orders_data['order_id'].values]
# no. of mismatch order ids
len(non_inc_ids)

278

In [38]:
# remove order payments data order id's that are not in order data order id's
idx_to_drop = []
for op_id in non_inc_ids:
    opid_idx = list(order_pay_data.query(f"order_id == '{op_id}'").index)
    for idx in opid_idx:
        if idx in idx_to_drop:
            continue
        else:
            idx_to_drop.append(idx)

In [40]:
# drop id's and save
order_pay_data.drop(idx_to_drop, inplace=True)
order_pay_data.to_csv('olist_order_payments_dataset.csv\olist_order_payments_dataset_dropped_dups_ids.csv', index=False, encoding='utf-8')

# Clean Order Items Data

In [13]:
# load data
order_items_data = pd.read_csv('olist_order_items_dataset.csv\olist_order_items_dataset.csv')
orders_data = pd.read_csv('olist_orders_dataset.csv\olist_orders_clean_dataset.csv')
products_data = pd.read_csv('olist_products_dataset.csv\olist_products_clean.csv')

In [12]:
order_items_data

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14
...,...,...,...,...,...,...,...
112645,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41
112646,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,2018-07-20 04:31:48,350.00,36.53
112647,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,2017-10-30 17:14:25,99.90,16.95
112648,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,2017-08-21 00:04:32,55.99,8.72


In [14]:
# check order ids in order items data that are not in orders data
oid_non_inc = [x for x in order_items_data['order_id'].values if x not in orders_data['order_id'].values]
len(oid_non_inc)

302

In [15]:
# remove order items data order id's that are not in order data order id's
idx_to_drop = []
for oi_id in oid_non_inc:
    oid_idx = list(order_items_data.query(f"order_id == '{oi_id}'").index)
    for idx in oid_idx:
        if idx in idx_to_drop:
            continue
        else:
            idx_to_drop.append(idx)

In [17]:
# drop id's and save
order_items_data.drop(idx_to_drop, inplace=True)
order_items_data.to_csv('olist_order_items_dataset.csv\olist_order_items_dataset_clean.csv', index=False, encoding='utf-8')

In [18]:
# check order items data product id's that are not in products data
oipd_non_inc = [x for x in order_items_data['product_id'].values if x not in products_data['product_id'].values]
len(oipd_non_inc)

0

In [8]:
# check for duplicated orders
order_items_data = pd.read_csv('olist_order_items_dataset.csv\olist_order_items_dataset_clean.csv')
order_items_data[order_items_data['order_id'].duplicated()].head(50)

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
14,0008288aa423d2a3f00fcb17cd7d8719,2,368c6c730842d78016ad823897a372db,1f50f920176fa81dab994f9023523100,2018-02-21 02:55:52,49.90,13.37
33,00143d0f86d6fbd9f9b38ab440ac16f5,2,e95ee6822b66ac6058e2e4aff656071a,a17f621c590ea0fab3d5d883e1630ec6,2017-10-20 16:07:52,21.33,15.10
34,00143d0f86d6fbd9f9b38ab440ac16f5,3,e95ee6822b66ac6058e2e4aff656071a,a17f621c590ea0fab3d5d883e1630ec6,2017-10-20 16:07:52,21.33,15.10
43,001ab0a7578dd66cd4b0a71f5b6e1e41,2,0b0172eb0fd18479d29c3bc122c058c2,5656537e588803a555b8eb41f07a944b,2018-01-04 02:33:42,24.89,17.63
44,001ab0a7578dd66cd4b0a71f5b6e1e41,3,0b0172eb0fd18479d29c3bc122c058c2,5656537e588803a555b8eb41f07a944b,2018-01-04 02:33:42,24.89,17.63
...,...,...,...,...,...,...,...
405,00f86b368251d739f1896d41469b2b7a,3,9b08a09bf2fd9731cfca8cb3db24457c,392353362d22cc2c236e1ee81ff19890,2018-08-20 23:30:08,39.97,18.38
412,00fb055886536063afa0dbee07a0944f,2,5d422bd54b54307ff7a5e770f40946e4,e88c9b79e592e370d6bd852eeefbf057,2017-06-02 02:25:13,11.90,16.79
420,00fed9e582d5a3e38566872eec164ac6,2,1491a8ff192d4b5c369b0eaa2fe33ee5,37be5a7c751166fbc5f8ccba4119e043,2017-10-03 17:07:16,179.99,26.10
437,010b143d83a59b355cd5a75c0f0fd785,2,6ff1fc9209c7854704a4f75c9fac41b4,4e922959ae960d389249c378d1c939f5,2017-09-05 23:44:47,22.00,34.15


In [12]:
# drop duplicates except last duplicated entry, orders are recorded sequentially increasing with number of items
order_items_data.drop_duplicates(subset='order_id', keep='last', inplace=True, ignore_index=True)

In [20]:
# save
order_items_data.to_csv('olist_order_items_dataset.csv\olist_order_items_dataset_clean.csv', index=False, encoding='utf-8')